In [ ]:
import pandas as pd
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import logging

# ========================
# CONFIG
# ========================

INPUT_CSV = "Data_clean/08.researchers_with_themes_expertise_openai.csv"
OUTPUT_CSV = "Data_clean/08.researchers_with_themes_expertise_cleaned.csv"

SIMILARITY_THRESHOLD = 85  # Fuzzy matching similarity threshold

# Set up logging
logging.basicConfig(level=logging.INFO)

# ========================
# DICTIONARIES
# (Now all keys/values are lowercase for consistent handling)
# ========================

SYNONYMS = {
    # Machine Learning
    "ml-based research": "machine learning",
    "machine learning algorithms": "machine learning",
    "large language models": "machine learning",
    "llms": "machine learning",
    "ml research": "machine learning",
    "machine-learning": "machine learning",

    # Neuroimaging
    "fnirs neuroimaging": "fnirs",
    "functional near-infrared spectroscopy (fnirs)": "fnirs",
    "functional near-infrared spectroscopy": "fnirs",
    "electroencephalography": "eeg",
    "fmri": "fmri",
    "functional magnetic resonance imaging": "fmri",
    "optical imaging": "neuroimaging",
    "imaging techniques": "neuroimaging",
    "cortical imaging": "neuroimaging",

    # Cross-linguistic
    "crosslinguistic evidence": "cross-linguistic evidence",
    "cross-linguistic comparison": "cross-linguistic comparison",
    "cross-linguistic research": "cross-linguistic research",

    # Eye Tracking
    "eye-tracking": "eye tracking",
    "eye tracking": "eye tracking",

    # Multimodal Communication
    "multimodal communication research": "multimodal communication",
    "multimodal research": "multimodal communication",

    # Bayesian
    "bayesian analyses": "bayesian analysis",
    "bayesian approach": "bayesian analysis",
    "bayesian inference": "bayesian analysis",
    "bayesian models": "bayesian analysis",

    # Deaf/Sign Language
    "sign languages": "sign language",
    "sign language": "sign language",
    "ngt": "sign language",
    "deaf community": "deaf communities",

    # Cochlear Implant
    "cochlear implantation": "cochlear implant",

    # GPT / ChatGPT
    "chatgpt": "chatgpt",
    "gpt": "chatgpt",

    # Variation of lexical usage
    "lexicography": "lexicography",
    "computational methods": "computational methods",
    "computational modeling": "computational modeling",

    # Random Forest
    "random forest models": "random forest",
    "random forest model": "random forest",

    # Observational Methods
    "observational study designs": "observational methods",

    # Emotional processes
    "emotional processes": "emotional processing",

    # DGS-LEX
    "dgs-lex": "dgs-lex",

    # DFG-Funded
    "dfg-funded research training group": "dfg-funded research training group",

    # PhD
    "phd thesis": "phd thesis",

    # Non-linguistic
    "non-linguistic deceptions": "nonlinguistic deceptions",
    "qud": "qud",
    "elan": "elan",
    "openpose": "openpose",
}

ACRONYMS = {
    "fmir": "fmri",
    "ml": "machine learning",
    "fmri": "fmri",
    "fnirs": "fnirs",
    "eeg": "eeg",
    "erps": "erps",
    "dgs-lex": "dgs-lex",
    "tms": "tms",
    "ai": "ai",
    "cnn": "convolutional neural networks",
    "rnn": "recurrent neural networks",
    "dfg-funded research training group": "dfg-funded research training group"
}

REMOVE_TERMS = {
    "language", "education", "philosophy",
    "psychology", "communication", "research methodology", "ai innovation",
    "information structure", "logic", "discourse analysis", "research focus",
    "studies", "areas of expertise", "experimental approach", "experimental investigations",
    "data analysis", "methodological frameworks", "research methodologies", "teaching",
    "translation", "programming", "therapy", "experimental methodologies", "ai innovation",
    "linguistics", "phd", "structure", "metaphor", "research contributions", "research opportunities", "research outcomes"

}

# ========================
# HELPER FUNCTIONS
# ========================

def normalize_keyword(kw: str) -> str:
    """
    1) Convert the keyword to lowercase, strip whitespace.
    """
    return kw.strip().lower()

def map_synonym(kw: str) -> str:
    """
    2) If the entire lowercase keyword is in SYNONYMS, replace it.
       Log the mapping if it happens.
    """
    if kw in SYNONYMS:
        mapped_value = SYNONYMS[kw]
        logging.info(f"[Synonym mapped] '{kw}' -> '{mapped_value}'")
        return mapped_value
    return kw

def enforce_acronyms(final_label: str) -> str:
    """
    3) If final_label is in ACRONYMS, replace it.
       Log the enforcement if it happens.
    """
    if final_label in ACRONYMS:
        new_label = ACRONYMS[final_label]
        logging.info(f"[Acronym enforced] '{final_label}' -> '{new_label}'")
        return new_label
    return final_label

def merge_similar_keywords(keywords, threshold=SIMILARITY_THRESHOLD):
    """
    4) Merge fuzzy-similar keywords.
       If similarity >= threshold, skip adding duplicates.
       Log whenever we skip a near-duplicate.
    """
    merged = []
    for kw in keywords:
        matched = False
        for existing in merged:
            ratio = fuzz.ratio(kw.lower(), existing.lower())
            if ratio >= threshold:
                logging.info(f"[Fuzzy merge] Skipped '{kw}' ~ '{existing}' (ratio={ratio})")
                matched = True
                break
        if not matched:
            merged.append(kw)
    return merged

def clean_themes_or_expertise(cell_value: str) -> str:
    """
    5) Clean and normalize a 'Themes' or 'Expertise' cell.
       - Split on multiple delimiters (; and ,).
       - Lowercase each keyword.
       - Map synonyms.
       - Enforce acronyms.
       - Use fuzzy dedup to merge near-duplicates.
       - Remove vague/forbidden terms.
       - Sort the final keywords alphabetically.
       - Log if we remove a term from REMOVE_TERMS.
    """
    if not isinstance(cell_value, str):
        return ""

    # ---- 1) Split on semicolon OR comma (add more if needed) ----
    #    We’ll split on either delimiter and accumulate tokens
    tokens_raw = re.split(r'[;,]', cell_value)  
    #   This returns all substrings split by either semicolon or comma
    items = [x.strip() for x in tokens_raw if x.strip()]

    # ---- 2) Normalize (lowercase, synonyms, acronyms) ----
    cleaned = []
    for it in items:
        it_norm = normalize_keyword(it)
        if not it_norm:
            continue
        
        # Synonyms
        mapped = map_synonym(it_norm)
        
        # Acronyms
        mapped_acr = enforce_acronyms(mapped)
        
        cleaned.append(mapped_acr)

    # ---- 3) Fuzzy duplicates and remove exact duplicates within the list ----
    # First, remove exact duplicates
    unique_cleaned = []
    for kw in cleaned:
        if kw not in unique_cleaned:
            unique_cleaned.append(kw)
        else:
            logging.info(f"[Exact duplicate removed] '{kw}'")

    # Then, fuzzy merge
    deduped = merge_similar_keywords(unique_cleaned, SIMILARITY_THRESHOLD)

    # ---- 4) Remove terms in REMOVE_TERMS ----
    final_filtered = []
    for term in deduped:
        if term.lower() in REMOVE_TERMS:
            logging.info(f"[Removed Term] '{term}' (in REMOVE_TERMS)")
        else:
            final_filtered.append(term)

    # ---- 5) Sort final keywords alphabetically for consistency ----
    final_filtered.sort()

    # ---- 6) Return them as a semicolon-separated string ----
    return "; ".join(final_filtered)

# ========================
# MAIN FUNCTION
# ========================
def main():
    df = pd.read_csv(INPUT_CSV)
    
    # Drop duplicates by 'Full Name'
    before_dedup = df.shape[0]
    df.drop_duplicates(subset="Full Name", keep="first", inplace=True)
    after_dedup = df.shape[0]
    logging.info(f"Removed {before_dedup - after_dedup} duplicate row(s) based on Full Name.")

    # Clean the Themes and Expertise columns
    df["Themes_Clean"] = df["Themes"].apply(clean_themes_or_expertise)
    df["Expertise_Clean"] = df["Expertise"].apply(clean_themes_or_expertise)
    
    # Save the cleaned dataframe
    df.to_csv(OUTPUT_CSV, index=False)
    logging.info(f"Saved cleaned file to '{OUTPUT_CSV}'.")

if __name__ == "__main__":
    main()

INFO:root:Removed 0 duplicate row(s) based on Full Name.
INFO:root:[Synonym mapped] 'lexicography' -> 'lexicography'
INFO:root:[Exact duplicate removed] 'digital scenes generation'
INFO:root:[Synonym mapped] 'multimodal communication research' -> 'multimodal communication'
INFO:root:[Removed Term] 'ai innovation' (in REMOVE_TERMS)
INFO:root:[Removed Term] 'information structure' (in REMOVE_TERMS)
INFO:root:[Removed Term] 'information structure' (in REMOVE_TERMS)
INFO:root:[Synonym mapped] 'multimodal communication research' -> 'multimodal communication'
INFO:root:[Removed Term] 'linguistics' (in REMOVE_TERMS)
INFO:root:[Removed Term] 'discourse analysis' (in REMOVE_TERMS)
INFO:root:[Removed Term] 'ai innovation' (in REMOVE_TERMS)
INFO:root:[Synonym mapped] 'sign language' -> 'sign language'
INFO:root:[Synonym mapped] 'sign languages' -> 'sign language'
INFO:root:[Synonym mapped] 'openpose' -> 'openpose'
INFO:root:[Exact duplicate removed] 'research projects'
INFO:root:[Exact duplicate 